In [1]:
# --- Bloco 1: Carregar Bibliotecas e Dados ---

# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import joblib
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Carregando os arquivos de dados que já processamos
try:
    df_sales = pd.read_parquet('../data/redis/sales.parquet')
    df_customers = pd.read_parquet('../data/redis/customers.parquet')
    print("Arquivos carregados com sucesso!")
except FileNotFoundError as e:
    print(f"Erro ao carregar o arquivo: {e}")

Arquivos carregados com sucesso!


In [2]:
# --- Bloco 2: Unificar Vendas e Segmentos ---
# Unifica os dados de vendas com os segmentos dos clientes
df_merged = pd.merge(
    df_sales,
    df_customers[['id_cliente', 'segmento']],
    on='id_cliente',
    how='left'
)
df_merged.dropna(subset=['segmento'], inplace=True)
print("\nDados unificados e prontos para a criação das recomendações.")


Dados unificados e prontos para a criação das recomendações.


In [3]:
# --- Bloco 3: Calcular o Trecho Mais Popular por Segmento (Baseline) ---
print("\nCalculando a recomendação de baseline...")
top_trecho_por_segmento = (
    # df_merged.groupby('segmento')['trecho_alias']
    df_merged.groupby('segmento')['produto']
    .value_counts()
    .groupby(level=0)
    .head(1)
    .reset_index(name='contagem')
)
mapa_baseline_alias = pd.Series(
    #top_trecho_por_segmento.trecho_alias.values, 
    top_trecho_por_segmento.produto.values, 
    index=top_trecho_por_segmento.segmento
).to_dict()
print("Recomendação de baseline calculada com sucesso.")



Calculando a recomendação de baseline...
Recomendação de baseline calculada com sucesso.


In [4]:
# --- Bloco 4: Construir Modelo com Matriz Esparsa (Otimizado) ---
print("\nIniciando a construção do modelo de recomendação personalizada (com matriz esparsa)...")

# 1. Mapear IDs e Trechos para Índices numéricos
# Criamos um índice numérico único para cada cliente e cada trecho
user_map = {id: i for i, id in enumerate(df_merged['id_cliente'].unique())}
# trecho_map = {trecho: i for i, trecho in enumerate(df_merged['trecho_alias'].unique())}
trecho_map = {trecho: i for i, trecho in enumerate(df_merged['produto'].unique())}

# Criamos os mapas inversos para traduzir de volta depois
inverse_trecho_map = {i: trecho for trecho, i in trecho_map.items()}

# Mapeamos os dados de vendas para os novos índices
df_merged['user_idx'] = df_merged['id_cliente'].map(user_map)
#df_merged['trecho_idx'] = df_merged['trecho_alias'].map(trecho_map)
df_merged['trecho_idx'] = df_merged['produto'].map(trecho_map)

# 2. Criar a Matriz Esparsa
# Contamos quantas vezes cada usuário comprou cada trecho
interactions = df_merged.groupby(['user_idx', 'trecho_idx']).size().reset_index(name='count')
# Criamos a matriz esparsa, que é extremamente eficiente em memória
sparse_matrix = csr_matrix((interactions['count'], (interactions['user_idx'], interactions['trecho_idx'])), 
                           shape=(len(user_map), len(trecho_map)))

print("Matriz esparsa de interações criada com sucesso!")

# 3. Treinar o Modelo TruncatedSVD
N_FATORES = 20
svd = TruncatedSVD(n_components=N_FATORES, random_state=42)
matriz_decomposta = svd.fit_transform(sparse_matrix)
print("Modelo SVD treinado.")

# 4. Reconstruir a Matriz de Previsões
matriz_previsoes = np.dot(matriz_decomposta, svd.components_)
print("Matriz de previsões reconstruída.")


Iniciando a construção do modelo de recomendação personalizada (com matriz esparsa)...
Matriz esparsa de interações criada com sucesso!
Modelo SVD treinado.
Matriz de previsões reconstruída.


In [5]:
#--- Bloco 5: Gerar Recomendações Finais e Salvar ---
print("\nIniciando a geração das recomendações finais...")

def gerar_top_recomendacoes(id_cliente, segmento_cliente):
    """ Gera a melhor recomendação para um cliente usando SVD ou baseline. """
    if id_cliente not in user_map:
        return mapa_baseline_alias.get(segmento_cliente, "Trecho em Oferta")

    user_idx = user_map[id_cliente]
    # Pega a linha de previsões para este usuário
    user_predictions = matriz_previsoes[user_idx, :]
    
    # Obtém os índices dos trechos que o usuário já comprou
    known_item_indices = sparse_matrix[user_idx, :].nonzero()[1]
    
    # Define as previsões para os itens já comprados como um valor muito baixo para que não sejam recomendados
    user_predictions[known_item_indices] = -999
    
    # Encontra o índice do trecho com a maior previsão
    top_item_index = np.argmax(user_predictions)
    
    # Converte o índice de volta para o nome do trecho
    return inverse_trecho_map[top_item_index]

# Aplica a função para gerar a recomendação para cada cliente.
df_customers['sugestao_prox_produto'] = df_customers.apply(
    lambda row: gerar_top_recomendacoes(row['id_cliente'], row['segmento']),
    axis=1
)

# --- SALVANDO O MODELO TREINADO E OS DADOS ---
caminho_modelo = '../data/redis/modelo_recomendacao_svd.joblib'
caminho_customers = '../data/redis/customers.parquet'

# Salva o modelo SVD e outros artefatos para uso futuro.
joblib.dump({
    'svd_model': svd,
    'user_map': user_map,
    'trecho_map': trecho_map,
    'inverse_trecho_map': inverse_trecho_map,
    'sparse_matrix': sparse_matrix
}, caminho_modelo)

# Salva o arquivo de clientes atualizado com as recomendações reais.
df_customers.to_parquet(caminho_customers, index=False)

print(f"\nModelo SVD e artefatos salvos em: {caminho_modelo}")
print(f"Arquivo 'customers.parquet' atualizado com sucesso com as recomendações!")
display(df_customers[['nome_cliente', 'segmento', 'sugestao_prox_produto']].head())


Iniciando a geração das recomendações finais...

Modelo SVD e artefatos salvos em: ../data/redis/modelo_recomendacao_svd.joblib
Arquivo 'customers.parquet' atualizado com sucesso com as recomendações!


,nome_cliente,segmento,sugestao_prox_produto
0,Hellena Freitas,Perdidos,Pires do Galho -> Jesus
1,Gael Henrique Viana,Fiéis,Melo -> Ramos
2,Lucca Abreu,Fiéis,Nogueira de Costa -> Rodrigues Alegre
3,Benicio Pereira,Fiéis,Melo -> Pires do Galho
4,Luana Lopes,Perdidos,Pires do Galho -> Albuquerque
